# Save and Load Models

## 데이터셋 불러오기

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.5.0


In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
print(train_images.shape)
print(train_images[0].shape)
print(train_images[0].reshape(-1, 28*28).shape)

(60000, 28, 28)
(28, 28)
(1, 784)


In [ ]:
# 모델 실행 속도를 빠르게 하기 위해 처음 1,000개만 사용
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 모델 구성

In [ ]:
# 간단한 sequential 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    
    return model

In [ ]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련 중 체크포인트 저장
`tf.keras.callbacks.ModelCheckpoint`를 사용하여 훈련 중간이나 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장할 수 있습니다.

### 체크포인트 콜백 사용

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# 콜백을 사용하여 모델 훈련
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 13ms/step - loss: 1.1655 - sparse_categorical_accuracy: 0.6660 - val_loss: 0.7055 - val_sparse_categorical_accuracy: 0.7860

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 0.4234 - sparse_categorical_accuracy: 0.8710 - val_loss: 0.5217 - val_sparse_categorical_accuracy: 0.8450

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2845 - sparse_categorical_accuracy: 0.9220 - val_loss: 0.4790 - val_sparse_categorical_accuracy: 0.8490

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 10ms/step - loss: 0.2016 - sparse_categorical_accuracy: 0.9550 - val_loss: 0.4829 - val_sparse_categorical_accuracy: 0.8430

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 0.1645 - sparse_ca

In [ ]:
# 체크포인트 파일이 생성되고 에포크가 종료될 때마다 업데이트됨
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [ ]:
# 기본 모델 생성 및 평가(1/10 수준)
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3651 - sparse_categorical_accuracy: 0.1220
Untrained model, accuracy: 12.20%


In [ ]:
# 가중치를 로드하고 재평가
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4182 - sparse_categorical_accuracy: 0.8650
Restored model, accuracy: 86.50%


### 체크포인트 콜백 옵션
콜백에는 체크포인트마다 고유한 이름을 붙이거나 체크포인트 저장 주기를 조정하는 기능이 있습니다.

In [ ]:
# 에포크 번호를 포함하여 파일명 설정
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# 5번째 에포크마다 가중치를 저장하는 콜백 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

In [ ]:
model = create_model()

# checkpoint_path 포맷을 사용하여 가중치 저장
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [ ]:
os.listdir(checkpoint_dir)

['cp-0010.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001']

In [ ]:
# 마지막 체크포인트 선택
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [ ]:
# 마지막 체크포인트를 로드하여 재평가
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4931 - sparse_categorical_accuracy: 0.8780
Restored model, accuracy: 87.80%


## 수동으로 가중치 저장

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4931 - sparse_categorical_accuracy: 0.8780
Restored model, accuracy: 87.80%


## 전체 모델 저장
- model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다.
- 모델을 저장하기 때문에 원본 파이썬 코드가 없어도 사용할 수 있습니다.
- 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.
- SavedModel 혹은 HDF5으로 모델을 저장할 수 있습니다.

### SavedModel format
SavedModel 포맷은 프로토콜 버퍼 이진 파일과 텐서플로 체크포인트를 담고 있는 디렉토리입니다.

In [ ]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1488 - sparse_categorical_accuracy: 0.6690
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4344 - sparse_categorical_accuracy: 0.8720
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2911 - sparse_categorical_accuracy: 0.9180
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2085 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1540 - sparse_categorical_accuracy: 0.9690


In [ ]:
# SavedModel로 전체 모델 저장 
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# 저장된 디렉토리 확인
!ls saved_model
!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [ ]:
# 모델 불러오기
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4506 - sparse_categorical_accuracy: 0.8550
Restored model, accuracy: 85.50%
(1000, 10)


### HDF5 format
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다. 케라스는 모델 구조를 파악하여 다음과 같은 모든 것을 저장합니다.

- 가중치 값
- 모델 구조
- 모델의 훈련 설정(컴파일할 때 전달한 값)
- 옵티마이저와 필요하면 옵티마이저 상태(중지된 지점부터 다시 시작할 수 있습니다)

In [ ]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.2369 - sparse_categorical_accuracy: 0.6240
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4414 - sparse_categorical_accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2840 - sparse_categorical_accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2168 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1682 - sparse_categorical_accuracy: 0.9660


In [ ]:
# 전체 모델을 HDF5 파일(.h5)로 저장
model.save('my_model.h5')

In [ ]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4397 - sparse_categorical_accuracy: 0.8580
Restored model, accuracy: 85.80%
